In [1]:
# Import necessary libraries
import base64
import json
import sys 
import os
import fitz
import requests
import re
from fitz import Document, Page, Rect

# Load double page

In [56]:
src = fitz.open("../PyMuPDF_playground/574_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(5)



# Extract category of double page

Create list of categories

In [76]:
categories = ['editorial', 'augustiner:in', 'einsicht', 'das wahre leben', 'cover', 'tun & lassen', 'vorstadt', 'lokalmatador:in nº', 'vorstadt', 'art.ist.in', 'dichter innenteil', 'augustinchen']

If same category, do not split into single pages.
If different categories, split couble page into two single pages

In [79]:
src = fitz.open("../PyMuPDF_playground/574_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(2)


rect = fitz.Rect(0, 0, page.rect.width, 60)
text = page.get_textbox(rect)
# Only remove all digits if lokalmatador is not in the text
if 'lokalmatador:in' not in text:
    # Remove all digits from the text
    result = ''.join(i for i in text if not i.isdigit())
    # Remove all \n from the text
    #result = result.replace(' ', '')
    result = result.lower()
    print(result)
else:
    char_index_beginning = text.find('lokalmatador:in')
    char_index_end = char_index_beginning + 22
    lokalmatadorin = text[char_index_beginning:char_index_end]
    text = text.replace(lokalmatadorin, '')
    # Remove all digits from the text
    result = ''.join(i for i in text if not i.isdigit())
    # Remove all \n from the text
    #result = result.replace(' ', '')
    result = result.lower()
    print(result)
i = 0
for word in result.split('\n'):
    word = word.strip() 
    if word in categories:
        i += 1

if i == 2:
    print('Both categories on top')
elif i == 1:
    print('One category on top')
    # Execute code below to extract category from left side
else:
    raise Exception('No category on top')





das wahre leben 

einsicht
Both categories on top


If in first print statement only one category has been found, check if category is written vertically on outer left side of page 

In [33]:
rect = fitz.Rect(0, 0, 50, page.rect.height)
text = page.get_text("rect", rect)
# Remove all digits from the text
result = ''.join(i for i in text if not i.isdigit())
# Remove all \n from the text
result = result.replace('\n', '')
result = result.lower()
result

'tun & lassen'

# Extract text from double page

In [167]:
src = fitz.open("../PyMuPDF_playground/570_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(3)
# Check for chars in different color than black and font size bigger than 50
def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)


# read page text as a dictionary, suppressing extra spaces in CJK fonts
blocks = page.get_text("dict", flags=11)["blocks"]
for b in blocks:  # iterate through the text blocks
    for l in b["lines"]:  # iterate through the text lines
        for s in l["spans"]:  # iterate through the text spans
            font_properties = "Font: '%s' (%s), size %g, color #%06x" % (
                s["font"],  # font name
                flags_decomposer(s["flags"]),  # readable font flags
                s["size"],  # font size
                s["color"],  # font color
            )
            # print("Text: '%s'" % s["text"])  # simple print of text
            # print(font_properties)
            # if case to search for a text start 
            colour_code = "#%06x" % (s["color"])
            if colour_code != "#2e2013" and colour_code != "#000000":
                if not s["text"].isdigit() and len(s["text"]) < 3 and s["text"] != "■":
                    print("Text: '%s'" % s["text"], font_properties)  
                    starting_words = s["text"]
            
            # TODO search for the word before the "■"
            # if case to search for a text end with "■"
            if colour_code != "#2e2013" and colour_code != "#000000" and s["text"] == "■":
                ending_words = s["text"]
                print("End: '%s'" % s["text"], s["color"])  # simple print of text

            # if case to search for a title
            # if s["flags"] & 2 ** 4:
            #     print("Title: '%s'" % s["text"], font_properties)

            if s["font"] == "AmasisMTStd-Bold":
                print("Main Title: '%s'" % s["text"], font_properties)

            if s["font"] == "Sentinel-Medium":
                print("Second Title of Excerpt: '%s'" % s["text"], font_properties)

            if s["font"] == "MyriadPro-Regular" and s["text"] not in categories:
                print("Excerpt: '%s'" % s["text"], font_properties)

            # if case to search for quotes 
            # TODO how to handle author name below the quote written in bold
            if s["flags"] & 2 ** 1 and s["font"] == "AmasisMTStd-Italic":
                print("Italic: '%s'" % s["text"], font_properties)



Text: '«E' Font: 'Sentinel-Book' (serifed, proportional), size 56.6946, color #ea405b
Main Title: 'Dem Frieden auf der Spur' Font: 'AmasisMTStd-Bold' (serifed, proportional, bold), size 36.8145, color #2e2013
Main Title: 'Die gewaltfreie Lösung von Konflikten ' Font: 'AmasisMTStd-Bold' (serifed, proportional, bold), size 11.6814, color #2e2013
Excerpt: ' ist eines der Hauptanliegen ' Font: 'MyriadPro-Regular' (serifed, proportional), size 10.3945, color #2e2013
Excerpt: 'der Friedensforschung. Als Forschungszweig ohne akademische Lobby führt ' Font: 'MyriadPro-Regular' (serifed, proportional), size 10.3945, color #2e2013
Excerpt: 'sie allerdings ein Schattendasein – selbst in der neutralen Alpenrepublik, der ' Font: 'MyriadPro-Regular' (serifed, proportional), size 10.3945, color #2e2013
Excerpt: 'sie eigentlich ein großes Anliegen sein sollte. ' Font: 'MyriadPro-Regular' (serifed, proportional), size 10.3945, color #2e2013
Excerpt: 'TEXT: CHRISTOPH FELLMER' Font: 'MyriadPro-Regular' (

# Angehensweise zur Extrahierung des Cover-Texts
1. Nimm den gesamten Text von `page.get_text("text")`
2. Bestimme Titel, Ausschnitt, Zitate, Bildunterschriften, Credits (Author, Foto etc.)
3. Lösche schrittweise Passagen aus dem Text, die nicht Teil des Inhalts sind (Titel, Ausschnitt...) -> Inhalt muss zwischen diesen Passagen liegen
4. Abhängig, ob auf der gleichen Seite schon der Text endet durch ein "■", muss der Text mit der zweiten Seite zusammengeknüpft werden
5. Sende das Cover Bild auf der linken Seite als Hauptbild mit und füge die restlich Bilder mit an

# Split double page in two single pages

In [ ]:
r = page.rect
d = fitz.Rect(
    # CropBox displacement if being set
    page.cropbox_position,
    page.cropbox_position,
)  # starting at (0, 0)

r1 = r
r1.x1 = r1.x1 / 2  # left side of double page
r2 = r1 + (r1.width, 0, r1.width, 0)  # right side of double page
# r2 = r + (r.width / 2, 0, 0, 0)  # right side of double page

r2 += d  # add the CropBox displacement
new_page_left = new_doc.new_page(-1, width=r1.width, height=r1.height)
new_page_left.show_pdf_page(
    new_page_left.rect,
    src,
    page.number,
    clip=r1,
)